In [1]:
import umap
import hdbscan
import numpy as np
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.cluster as cluster
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
# Dimension reduction and clustering libraries
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
sns.set(context= 'notebook', style= 'white', rc={'figure.figsize':(14,10)})
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\Raw_And_More
df = pd.read_csv ("beta_post_Funnorm_PPMI_EPICn524final030618.csv")
df.head(2)

Z:\PPMI_Data\Excels\Raw_And_More


,200973410159_R03C01,200991620021_R01C01,200991620021_R03C01,200989060105_R03C01,200989060135_R07C01,200989060162_R04C01,200973410167_R04C01,200973410143_R01C01,200973410143_R06C01,200973410167_R05C01,...,200989060089_R02C01,200989060089_R01C01,200989060089_R04C01,200973410143_R08C01,200973410143_R07C01,200989060089_R03C01,200989060150_R04C01,200989060150_R05C01,200989060150_R06C01,200991620049_R08C01
cg14817997,0.834027,0.849201,0.772995,0.885288,0.772547,0.756429,0.719766,0.775910,0.768914,0.759352,...,0.788239,0.838946,0.845263,0.770405,0.852917,0.783813,0.806930,0.875691,0.661276,0.737917
cg26928153,0.945589,0.948538,0.962199,0.944048,0.926306,0.956061,0.912988,0.955983,0.957120,0.897589,...,0.951206,0.958975,0.957062,0.954738,0.947765,0.958086,0.948044,0.942405,0.954990,0.946272


In [3]:
df.index.name ='CpG'
df.reset_index(inplace=True)
print(df.shape)
df.head(2)

(864067, 525)


,CpG,200973410159_R03C01,200991620021_R01C01,200991620021_R03C01,200989060105_R03C01,200989060135_R07C01,200989060162_R04C01,200973410167_R04C01,200973410143_R01C01,200973410143_R06C01,...,200989060089_R02C01,200989060089_R01C01,200989060089_R04C01,200973410143_R08C01,200973410143_R07C01,200989060089_R03C01,200989060150_R04C01,200989060150_R05C01,200989060150_R06C01,200991620049_R08C01
0,cg14817997,0.834027,0.849201,0.772995,0.885288,0.772547,0.756429,0.719766,0.775910,0.768914,...,0.788239,0.838946,0.845263,0.770405,0.852917,0.783813,0.806930,0.875691,0.661276,0.737917
1,cg26928153,0.945589,0.948538,0.962199,0.944048,0.926306,0.956061,0.912988,0.955983,0.957120,...,0.951206,0.958975,0.957062,0.954738,0.947765,0.958086,0.948044,0.942405,0.954990,0.946272


In [4]:
df1 = pd.read_csv ("CpG_List_HavingCorres_GeneChr.csv")
df = pd.merge (df1, df, on= "CpG", how= "inner")
print(df.shape)
df.head(2)

(837962, 525)


,CpG,200973410159_R03C01,200991620021_R01C01,200991620021_R03C01,200989060105_R03C01,200989060135_R07C01,200989060162_R04C01,200973410167_R04C01,200973410143_R01C01,200973410143_R06C01,...,200989060089_R02C01,200989060089_R01C01,200989060089_R04C01,200973410143_R08C01,200973410143_R07C01,200989060089_R03C01,200989060150_R04C01,200989060150_R05C01,200989060150_R06C01,200991620049_R08C01
0,cg18478105,0.012578,0.014327,0.015461,0.010655,0.009996,0.009345,0.013387,0.010906,0.008541,...,0.010377,0.016028,0.009028,0.010601,0.011051,0.010528,0.011086,0.011627,0.012368,0.013576
1,cg09835024,0.022227,0.029524,0.027406,0.021273,0.022834,0.044152,0.038306,0.016127,0.015281,...,0.019746,0.025645,0.022651,0.022760,0.021201,0.021772,0.024651,0.017504,0.019783,0.023908


In [5]:
dft = df.transpose()

In [6]:
dft.rename(columns=dft.iloc[0], inplace = True)

In [7]:
dft.index.name ='Sentrix'
dft.reset_index(inplace=True)
dft.head(2)

,Sentrix,cg18478105,cg09835024,cg14361672,cg01763666,cg12950382,cg02115394,cg25813447,cg07779434,cg13417420,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
0,CpG,cg18478105,cg09835024,cg14361672,cg01763666,cg12950382,cg02115394,cg25813447,cg07779434,cg13417420,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
1,200973410159_R03C01,0.012578,0.022227,0.919272,0.886308,0.504573,0.035614,0.356011,0.360377,0.019089,...,0.806559,0.469303,0.371767,0.838957,0.929768,0.887072,0.91134,0.864708,0.061452,0.695212


In [8]:
dft = dft.drop ([0], axis = 0)
dft.head(2)

,Sentrix,cg18478105,cg09835024,cg14361672,cg01763666,cg12950382,cg02115394,cg25813447,cg07779434,cg13417420,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
1,200973410159_R03C01,0.012578,0.022227,0.919272,0.886308,0.504573,0.035614,0.356011,0.360377,0.019089,...,0.806559,0.469303,0.371767,0.838957,0.929768,0.887072,0.91134,0.864708,0.061452,0.695212
2,200991620021_R01C01,0.014327,0.029524,0.902918,0.883315,0.592818,0.024543,0.032831,0.016275,0.024952,...,0.808755,0.460212,0.397015,0.810033,0.90079,0.890029,0.91342,0.820941,0.116707,0.61726


In [9]:
df2 = pd.read_csv ("Sentrix_PATNO_APPRDX_Gender_of524Methyl_APP1_2.csv")
df2.head(2)

,Sentrix,PATNO,Gender,APPRDX
0,200991620021_R01C01,3001,1,1
1,200973410167_R04C01,3007,1,1


In [10]:
df3 = pd.merge (df2, dft, on= "Sentrix", how= "inner")
print(df3.shape)
df3.head(2)

(457, 837966)


,Sentrix,PATNO,Gender,APPRDX,cg18478105,cg09835024,cg14361672,cg01763666,cg12950382,cg02115394,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
0,200991620021_R01C01,3001,1,1,0.014327,0.029524,0.902918,0.883315,0.592818,0.024543,...,0.808755,0.460212,0.397015,0.810033,0.90079,0.890029,0.91342,0.820941,0.116707,0.61726
1,200973410167_R04C01,3007,1,1,0.013387,0.038306,0.899046,0.897997,0.776989,0.042385,...,0.75258,0.509289,0.464598,0.828355,0.886257,0.874093,0.895796,0.87725,0.057642,0.667532


## Male

In [11]:
df4 = df3[df3.Gender == 1]

In [12]:
df4.to_csv('Male_Chromosome.csv', index = False)

## Female

In [13]:
df4 = df3[df3.Gender == 2]

In [14]:
df4.to_csv('Female_Chromosome.csv', index = False)